<h1 align=center> Regression Models with Keras</font></h1>

## Introducción

Este notebook muestra un tutorial de como utilizar la libreria Keras para modelos de Regresión. 

- A pesar de la popularidad de bibliotecas más poderosas como PyToch y TensorFlow, no son fáciles de usar y tienen una curva de aprendizaje pronunciada. Por lo tanto, para las personas que recién están comenzando a aprender el aprendizaje profundo, no hay mejor biblioteca para usar que la biblioteca de Keras.

- keras es una API de alto nivel para construir modelos de Deep Learning. Keras ha ganado popularidad por su facilidad de uso y simplicidad sintáctica que facilita el desarrollo rápido. 

- La construcción de modelos de Deep Learning muy compleja se puede hacer con Keras con solo unas pocas lineas de código. Comenzemos

## Modelos de Regresion con Keras

### Objetivo de este notebook

<h5> 1. Como usar la librerias Keras para    
         construir un Modelo de Regresión.</h5>
<h5> 2. Descargar y limpiar el Dataset </h5>
<h5> 3. Construir una Red neuronal </h5>
<h5> 4. Entrenar y evaluar la Red. </h5> 

---

## Descargar y limpiar el Dataset

Comenzamos por importar las librerias Pandas y Numpy.

In [1]:
import pandas as pd
import numpy as np

<strong>El dataset trata sobre la resistencia a la compresión de diferentes muestras de hormigón en función de los volúmenes de los diferentes ingredientes que se utilizaron para fabricarlas. Los ingredientes incluyen:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>

- Descargamos el dataset y lo ubicamos en el directorio del proyecto.

In [2]:
concrete_data = pd.read_csv("concrete_data.csv")
concrete_data.head()

,cement,blast_furnace_slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age,concrete_compressive_strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


- Como se puede ver la primera muestra de concreto tiene 540 metros cubicos de cemento, 0 metros cubicos de blast furnace slag , 0 metros cubicos de fly ash, 162 metros cubicos de water, 2.5 metros cubicos de superplaticizer, 1040 metros cubicos de coarse aggregate, 676 metros cubicos de fine aggregate, etc. 

## Comprobemos cuantos puntos de datos tenemos

In [3]:
concrete_data.shape

(1030, 9)

- Asi que, hay aproximadamente 1000 muestras para entrenar nuestro modelo. Debido a las pocas muestras, debemos tener cuidado de no sobreajustar el entrenamiento de datos. 

- Revisemos el dataset en busca de valores faltantes.

In [4]:
concrete_data.describe()

,cement,blast_furnace_slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age,concrete_compressive_strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
concrete_data.isnull().sum()

cement                           0
blast_furnace_slag               0
fly_ash                          0
water                            0
superplasticizer                 0
coarse_aggregate                 0
fine_aggregate                   0
age                              0
concrete_compressive_strength    0
dtype: int64

- Los datos parecen muy limpios y listos para usarse y construir nuestro modelo

--- 

## Dividir los datos en *Predictors* y *Target*

La variable objetivo en este problema es la resistencia de la muestra de hormigón **(Strength)**. Por lo tanto, nuestros predictores serán todas las demás columnas.

In [6]:
concrete_data.columns

Index(['cement', 'blast_furnace_slag', 'fly_ash', 'water', 'superplasticizer',
       'coarse_aggregate', 'fine_aggregate ', 'age',
       'concrete_compressive_strength'],
      dtype='object')

- Por simplicidad cambiaremos el nombre de la Columna **concrete_compressive_strength** a **Strength**

In [9]:
concrete_data = concrete_data.rename(columns={"concrete_compressive_strength": "Strength"})

In [10]:
concrete_data.columns

Index(['cement', 'blast_furnace_slag', 'fly_ash', 'water', 'superplasticizer',
       'coarse_aggregate', 'fine_aggregate ', 'age', 'Strength'],
      dtype='object')

In [11]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # Todas las columnas excepto Strength
target = concrete_data['Strength'] # Columna Strength 

- Hagamos una revision rapida de los predictors y target en el dataframe

In [12]:
predictors.head()

,cement,blast_furnace_slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [13]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

- Finalmente, el último paso es normalizar los datos restando la media y dividiendo por la desviación estándar.

In [14]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,cement,blast_furnace_slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [16]:
predictors_norm

,cement,blast_furnace_slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069
...,...,...,...,...,...,...,...,...
1025,-0.045623,0.487998,0.564271,-0.092126,0.451190,-1.322363,-0.065861,-0.279597
1026,0.392628,-0.856472,0.959602,0.675872,0.702285,-1.993711,0.496651,-0.279597
1027,-1.269472,0.759210,0.850222,0.521336,-0.017520,-1.035561,0.080068,-0.279597
1028,-1.168042,1.307430,-0.846733,-0.279443,0.852942,0.214537,0.191074,-0.279597


- Vamos a guardar el numero de predictores en *n_cols* ya que necesitaremos este número al construir nuestra red.

In [15]:
n_cols = predictors_norm.shape[1] # numero de Predictores 

---

## Importando KERAS

- Para poder usar la biblioteca de Keras, primero deberá instalar TensorFlow.

In [17]:
import keras

- Ahora vamos a importar el resto de los paquetes desde la libreria Keras que necesitaremos para construir nuestro Modelo de Regresion

In [18]:
from keras.models import Sequential
from keras.layers import Dense

## Construir una Red Neuronal

- Definamos una función que defina nuestro modelo de regresión para que podamos llamarlo convenientemente para crear nuestro modelo.

In [21]:
def regression_model():
    # crear modelo
    model = Sequential()
    model.add(Dense(50, activation='relu',input_shape=(n_cols,)))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    
    # compilar el modelo
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

- La función *regression_model* crea un modelo que tiene 2 capas ocultas, cada una de 50 unidades ocultas.

## Entrenar y evaluar la Red

Llamamos a la funcion para crear nuestro modelo

In [22]:
# construir el modelo

model = regression_model()

- A continuación, entrenaremos y probaremos el modelo al mismo tiempo utilizando el método de ajuste. Dejaremos fuera el 30% de los datos para la validación y entrenaremos el modelo durante 100 épocas.

In [23]:
#Ajustar el modelo
model.fit(predictors_norm, target, validation_split=0.3, epochs=100, verbose=2)

Epoch 1/100
23/23 - 1s - loss: 1617.0205 - val_loss: 1090.0900
Epoch 2/100
23/23 - 0s - loss: 1440.6874 - val_loss: 921.3813
Epoch 3/100
23/23 - 0s - loss: 1149.7424 - val_loss: 678.3052
Epoch 4/100
23/23 - 0s - loss: 763.8055 - val_loss: 421.0351
Epoch 5/100
23/23 - 0s - loss: 420.9468 - val_loss: 250.0421
Epoch 6/100
23/23 - 0s - loss: 268.6611 - val_loss: 206.2218
Epoch 7/100
23/23 - 0s - loss: 238.4027 - val_loss: 204.5732
Epoch 8/100
23/23 - 0s - loss: 225.5851 - val_loss: 195.5253
Epoch 9/100
23/23 - 0s - loss: 215.3380 - val_loss: 193.8385
Epoch 10/100
23/23 - 0s - loss: 206.2601 - val_loss: 187.3731
Epoch 11/100
23/23 - 0s - loss: 198.5692 - val_loss: 182.6343
Epoch 12/100
23/23 - 0s - loss: 192.0274 - val_loss: 179.1884
Epoch 13/100
23/23 - 0s - loss: 186.9701 - val_loss: 175.6069
Epoch 14/100
23/23 - 0s - loss: 180.2624 - val_loss: 171.9978
Epoch 15/100
23/23 - 0s - loss: 175.5569 - val_loss: 170.2770
Epoch 16/100
23/23 - 0s - loss: 171.9935 - val_loss: 165.0474
Epoch 17/100


- Esto concluye esta guia de como construir un modelo de Regresion con una Red Neuronal en la libreria Keras, Siéntase libre de variar lo siguiente y observe el impacto que tiene cada cambio en el rendimiento del modelo:

1. Aumentar o disminuir la cantidad de neuronas en capas ocultas.
2. Agrega más capas ocultas
3. Aumentar el número de épocas epochs